# Ejercicio con dataset de Star Wars

## Inicialización del entorno

In [2]:
from pyspark.sql import *

spark = SparkSession.builder.master("local").appName("NOMBRE").getOrCreate()

22/05/25 07:55:57 WARN Utils: Your hostname, fjortega-desktop resolves to a loopback address: 127.0.1.1; using 150.214.188.52 instead (on interface enp6s0)
22/05/25 07:55:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/05/25 07:55:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## Cargar los datos de un archivo CSV en un Dataframe

In [3]:
df = spark.read.csv("./characters.csv", header=True)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- height: string (nullable = true)
 |-- mass: string (nullable = true)
 |-- hair_color: string (nullable = true)
 |-- skin_color: string (nullable = true)
 |-- eye_color: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- homeworld: string (nullable = true)
 |-- species: string (nullable = true)

+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|                name|height| mass|   hair_color|      skin_color|eye_color|birth_year|       gender| homeworld|       species|
+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|      Luke Skywalker|   172|   77|        blond|            fair|     blue|     19BBY|         male|  Tatooine|         Human|
|               C-3PO|   167|   75|           NA|            gold|   yellow|    112BBY|

## Seleccionar los personajes que no sean hombres o mujeres

### Usando una consulta SQL "en crudo"

In [4]:
df.createOrReplaceTempView("Personajes")
df2 = spark.sql("SELECT * FROM Personajes WHERE gender NOT LIKE '%male' ORDER BY mass")
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)
 |-- height: string (nullable = true)
 |-- mass: string (nullable = true)
 |-- hair_color: string (nullable = true)
 |-- skin_color: string (nullable = true)
 |-- eye_color: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- homeworld: string (nullable = true)
 |-- species: string (nullable = true)

+--------------------+------+-----+----------+----------------+---------+----------+-------------+---------+-------+
|                name|height| mass|hair_color|      skin_color|eye_color|birth_year|       gender|homeworld|species|
+--------------------+------+-----+----------+----------------+---------+----------+-------------+---------+-------+
|Jabba Desilijic T...|   175|1,358|        NA|green-tan, brown|   orange|    600BBY|hermaphrodite|Nal Hutta|   Hutt|
|               IG-88|   200|  140|      none|           metal|      red|     15BBY|         none|       NA|  Droid|
|             

### Usando funciones de SparkSQL

In [5]:
df2 = df.select(df.name).where(~df.gender.isin("male", "female"))
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)

+--------------------+
|                name|
+--------------------+
|               C-3PO|
|               R2-D2|
|               R5-D4|
|Jabba Desilijic T...|
|               IG-88|
|                 BB8|
+--------------------+



## Seleccionar personajes de un planeta determinado

### Usando una consulta SQL "en crudo"

In [6]:
df2 = spark.sql("SELECT name FROM Personajes WHERE homeworld LIKE 'Naboo'")
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)

+-------------+
|         name|
+-------------+
|        R2-D2|
|    Palpatine|
|Jar Jar Binks|
| Roos Tarpals|
|   Rugor Nass|
|     Ric Olié|
|Quarsh Panaka|
| Gregar Typho|
|        Cordé|
|        Dormé|
|Padmé Amidala|
+-------------+



### Usando funciones de SparkSQL

In [7]:
df2 = df.select(df.name).filter(df.homeworld == 'Naboo')
df2.printSchema()
df2.show()

root
 |-- name: string (nullable = true)

+-------------+
|         name|
+-------------+
|        R2-D2|
|    Palpatine|
|Jar Jar Binks|
| Roos Tarpals|
|   Rugor Nass|
|     Ric Olié|
|Quarsh Panaka|
| Gregar Typho|
|        Cordé|
|        Dormé|
|Padmé Amidala|
+-------------+



## Agregaciones

### Número de personajes originarios de cada planeta

In [8]:
df3 = df.select(df.name, df.homeworld).groupBy(df.homeworld).count()
df3.orderBy("count", ascending=False).show()


+-----------+-----+
|  homeworld|count|
+-----------+-----+
|      Naboo|   11|
|         NA|   10|
|   Tatooine|   10|
|  Coruscant|    3|
|   Alderaan|    3|
|     Kamino|    3|
|   Kashyyyk|    2|
|   Corellia|    2|
|     Ryloth|    2|
|     Mirial|    2|
|  Chandrila|    1|
|    Iktotch|    1|
|     Utapau|    1|
|Aleen Minor|    1|
|    Serenno|    1|
|      Shili|    1|
| Haruun Kal|    1|
|      Endor|    1|
|   Mon Cala|    1|
|   Champala|    1|
+-----------+-----+
only showing top 20 rows



### Número de especies originarias de cada planeta

In [9]:
df3 = df.select(df.species, df.homeworld).groupBy(df.homeworld, df.species).count()
df3.orderBy("homeworld","count", ascending=[True,False]).show()

+--------------+----------+-----+
|     homeworld|   species|count|
+--------------+----------+-----+
|      Alderaan|     Human|    3|
|   Aleen Minor|    Aleena|    1|
|        Bespin|     Human|    1|
|    Bestine IV|     Human|    1|
|Cato Neimoidia| Neimodian|    1|
|         Cerea|    Cerean|    1|
|      Champala|  Chagrian|    1|
|     Chandrila|     Human|    1|
|  Concord Dawn|     Human|    1|
|      Corellia|     Human|    2|
|     Coruscant|     Human|    2|
|     Coruscant|Tholothian|    1|
|      Dathomir|    Zabrak|    1|
|         Dorin|   Kel Dor|    1|
|         Endor|      Ewok|    1|
|        Eriadu|     Human|    1|
|      Geonosis| Geonosian|    1|
|   Glee Anselm|  Nautolan|    1|
|    Haruun Kal|     Human|    1|
|       Iktotch|  Iktotchi|    1|
+--------------+----------+-----+
only showing top 20 rows



## Manejo de RDDs

### Obtener RDD con nombres y alturas de personajes

In [10]:
rdd= df.select(df.name, df.height).rdd
print(rdd.take(20))

rdd2 = rdd.filter(lambda x:x[1]!="NA").map(lambda x: (x[0], int(x[1])))
print(rdd2.take(20))

[Row(name='Luke Skywalker', height='172'), Row(name='C-3PO', height='167'), Row(name='R2-D2', height='96'), Row(name='Darth Vader', height='202'), Row(name='Leia Organa', height='150'), Row(name='Owen Lars', height='178'), Row(name='Beru Whitesun lars', height='165'), Row(name='R5-D4', height='97'), Row(name='Biggs Darklighter', height='183'), Row(name='Obi-Wan Kenobi', height='182'), Row(name='Anakin Skywalker', height='188'), Row(name='Wilhuff Tarkin', height='180'), Row(name='Chewbacca', height='228'), Row(name='Han Solo', height='180'), Row(name='Greedo', height='173'), Row(name='Jabba Desilijic Tiure', height='175'), Row(name='Wedge Antilles', height='170'), Row(name='Jek Tono Porkins', height='180'), Row(name='Yoda', height='66'), Row(name='Palpatine', height='170')]
[('Luke Skywalker', 172), ('C-3PO', 167), ('R2-D2', 96), ('Darth Vader', 202), ('Leia Organa', 150), ('Owen Lars', 178), ('Beru Whitesun lars', 165), ('R5-D4', 97), ('Biggs Darklighter', 183), ('Obi-Wan Kenobi', 182)

### Media de estatura de los personajes

In [11]:
rdd2.map(lambda x: x[1]).mean()

174.35802469135803

### Estatura máxima de los personajes

In [12]:
rdd2.map(lambda x: x[1]).max()

264

In [18]:
df.select(df.name).where(df.height=='NA').count()

6

## Inferencia de tipos en DataFrames con PySpark

### Primero vamos a hacer un ligero preprocesado del dataset original, para que nos sirva de ejemplo

In [27]:
from pyspark.sql.functions import when,regexp_replace

df_edit = df.withColumn('height', 
    when(df.height=='NA',regexp_replace(df.height,'NA','-1')).otherwise(df.height))

df_edit.write.option("header",True).csv("characters_edit")

#### Pregunta: ¿Qué hemos hecho?

### Ahora lo cargamos y trabajamos con él... pero infiriendo el tipo de las columnas:

In [28]:
df = spark.read.csv("./characters_edit/*", header=True, inferSchema=True)
df.printSchema()
df.show()

root
 |-- name: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- mass: string (nullable = true)
 |-- hair_color: string (nullable = true)
 |-- skin_color: string (nullable = true)
 |-- eye_color: string (nullable = true)
 |-- birth_year: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- homeworld: string (nullable = true)
 |-- species: string (nullable = true)

+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|                name|height| mass|   hair_color|      skin_color|eye_color|birth_year|       gender| homeworld|       species|
+--------------------+------+-----+-------------+----------------+---------+----------+-------------+----------+--------------+
|      Luke Skywalker|   172|   77|        blond|            fair|     blue|     19BBY|         male|  Tatooine|         Human|
|               C-3PO|   167|   75|           NA|            gold|   yellow|    112BBY